#### Use titanic dataset. Handle the null values and convert the categorical values into numerical values. Make a classification model using Logistic Regression to predict the survival of a passenger on the ship.

In [1]:
import numpy as np
import pandas as pd


In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/rahul96rajan/sample_datasets/master/titanic.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
data.shape

(891, 12)

In [4]:
data.drop(labels=['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin'], axis=1, inplace=True)

In [5]:
data.describe()

,Survived,Pclass,Age,SibSp,Parch
count,891.000000,891.000000,714.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594
std,0.486592,0.836071,14.526497,1.102743,0.806057
min,0.000000,1.000000,0.420000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000
50%,0.000000,3.000000,28.000000,0.000000,0.000000
75%,1.000000,3.000000,38.000000,1.000000,0.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000


In [6]:
data.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Embarked      2
dtype: int64

In [7]:
data.dropna(subset=['Embarked'], inplace=True)

In [8]:
X = data.drop(labels='Survived', axis=1)
y = data['Survived']

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [10]:
X_train['Age'].value_counts()

30.00    22
24.00    22
18.00    21
25.00    20
28.00    18
         ..
12.00     1
32.50     1
70.50     1
20.50     1
0.42      1
Name: Age, Length: 84, dtype: int64

In [11]:
X_train[X_train['Age'].isna()]

,Pclass,Sex,Age,SibSp,Parch,Embarked
126,3,male,NaN,0,0,Q
109,3,female,NaN,1,0,Q
578,3,female,NaN,1,0,C
888,3,female,NaN,1,2,S
613,3,male,NaN,0,0,Q
...,...,...,...,...,...,...
358,3,female,NaN,0,0,Q
277,2,male,NaN,0,0,S
584,3,male,NaN,0,0,C
849,1,female,NaN,1,0,C


In [12]:
X_train.isna().sum()

Pclass        0
Sex           0
Age         129
SibSp         0
Parch         0
Embarked      0
dtype: int64

In [13]:
y_train[X_train['Age'].isnull()].sum() 
# i.e; 36 outof 129 poeple with missing age survived

36

In [14]:
print('Mode:({0}),  Median:{1},'
      '  Mean:{2}'.format(X_train['Age'].mode().values, X_train['Age'].median(),
                        X_train['Age'].mean()))

Mode:([24. 30.]),  Median:29.0,  Mean:29.754040968342643


In [15]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

i_median = SimpleImputer(strategy='median')
i_median.fit(X_train['Age'].values.reshape(-1, 1))

leber = LabelEncoder()
leber.fit(X_train['Sex'].values.reshape(-1, 1).ravel())

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X_train[['Embarked']])

def preprocess(data):
    df = data.copy()
    df.loc[:, 'Age'] = i_median.transform(df['Age'].values.reshape(-1,1))
    df.loc[:, 'Sex'] = leber.transform(df['Sex'].values.reshape(-1, 1).ravel())
    ohed_pd = pd.DataFrame(enc.transform(df[['Embarked']]).toarray(),
                           columns=['Emb_S','Emb_Q','Emb_S'], index=df.index)
    df = df.join(ohed_pd)
    df['kins'] = df['SibSp'] + df['Parch']
    df.drop(labels=['Embarked','SibSp','Parch'], axis=1, inplace=True)
    return df


In [16]:
# Pre-Processing Training data
X_train = preprocess(X_train)

In [17]:
# Using GridSearch with LeaveOneOut Cross validation to get descent parameters.
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
lg = LogisticRegression(solver='liblinear')

param_grid = [{'penalty': ['l1', 'l2'], 'C': [0.1, 1, 10]}]

grid_search = GridSearchCV(lg, param_grid, cv=loo, scoring='accuracy',
                           return_train_score=True)
grid_search.fit(X_train, y_train)
grid_search.best_estimator_

LogisticRegression(C=1, penalty='l1', solver='liblinear')

In [18]:
# Training using the best_estimators_
lg = LogisticRegression(C=1, penalty='l1', solver='liblinear')
lg.fit(X_train, y_train)

X_test = preprocess(X_test)
y_pred = lg.predict(X_test)
y_pred_train = lg.predict(X_train)

In [19]:
from sklearn.metrics import accuracy_score

print("Accuracy score(Test) : ", accuracy_score(y_test, y_pred))
print("Accuracy score(Train) : ", accuracy_score(y_train, y_train))

Accuracy score(Test) :  0.8340807174887892
Accuracy score(Train) :  1.0
